In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm

In [7]:
df = pd.read_csv("data/data.txt")
df = df.drop(columns='Unnamed: 0')
locations = df["location"].unique()
print(locations)

['Albania' 'Algeria' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahrain' 'Bangladesh' 'Barbados' 'Belgium' 'Bolivia'
 'Botswana' 'Brazil' 'Bulgaria' 'Cambodia' 'Cameroon' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Denmark'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Estonia' 'Ethiopia'
 'Finland' 'France' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Guatemala'
 'Honduras' 'Hong Kong' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kazakhstan'
 'Kenya' 'Kuwait' 'Latvia' 'Lebanon' 'Lithuania' 'Luxembourg' 'Malaysia'
 'Malta' 'Mauritius' 'Mexico' 'Moldova' 'Mongolia' 'Montenegro' 'Morocco'
 'Myanmar' 'Nepal' 'Netherlands' 'New Zealand' 'Nicaragua' 'Nigeria'
 'North Macedonia' 'Norway' 'Oman' 'Pakistan' 'Panama' 'Paraguay' 'Peru'
 'Philippines' 'Poland' 'Portugal' 'Qatar' 'Romania' 'Russia' 'Rwanda'
 'Saudi Arabia' 'Senegal' 'Serbia' 'Seychelles' 'Singapore' 'Slovakia'
 'Slovenia' 'South A

In [3]:
def get_results_from_query(url):
    results = []
    response = requests.get(url).json()

    if response and response.get("status") != "OK":
        print(response.get("status"))
    else:
        results = response["results"]

        return (response, results)

In [4]:
def get_next_results_from_query(response, url, results):
    if "next_page_token" in response:
        next_page_token = response["next_page_token"]
        next_page_url = url + f"&pagetoken{next_page_token}"
        response, next_results = get_results_from_query(next_page_url)

        if len(next_results) != 0:
            results.extend(next_results)
        
        return response, results

In [5]:
api_key = "redacted"
places_of_interest = {}

for i in tqdm(range(len(locations))):
    results = []
    location = locations[i]
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={location}+top+sights&language=en&key={api_key}"

    # bit janky but oh well capped at 60 results
    try:
        response, results = get_results_from_query(url)
    except:
        pass
    try:
        response, results = get_next_results_from_query(response, url, results)
    except:
        pass
    try:
        response, results = get_next_results_from_query(response, url, results)
    except:
        pass
    
    places_of_interest[location] = results

100%|██████████| 116/116 [03:21<00:00,  1.74s/it]


In [6]:
with open('poi.json', 'w') as fp:
    json.dump(places_of_interest, fp)